##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Examining the TensorFlow Graph

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tensorboard/graphs"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/graphs.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tensorboard/blob/master/docs/graphs.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/tensorboard/docs/graphs.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Overview

TensorBoard’s **Graphs dashboard** is a powerful tool for examining your TensorFlow model. You can quickly view a conceptual graph of your model’s structure and ensure it matches your intended design. You can also view a op-level graph to understand how TensorFlow understands your program. Examining the op-level graph can give you insight as to how to change your model. For example, you can redesign your model if training is progressing slower than expected.

This tutorial presents a quick overview of how to generate graph diagnostic data and visualize it in TensorBoard’s Graphs dashboard. You’ll define and train a simple Keras Sequential model for the Fashion-MNIST dataset and learn how to log and examine your model graphs. You will also use a tracing API to generate graph data for functions created using the new `tf.function` annotation.

## Setup

In [2]:
# %pip install tensorboard_plugin_profile

In [3]:

# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [4]:
from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

2025-05-23 22:36:21.987313: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-23 22:36:21.994296: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748054182.002885  251841 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748054182.005282  251841 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748054182.011634  251841 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

TensorFlow version:  2.19.0


In [5]:
import tensorboard
tensorboard.__version__

'2.19.0'

In [6]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

## Define a Keras model

In this example, the classifier is a simple four-layer Sequential model.

In [7]:
# Define the model.
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

/home/oleg/Developer/GitHub/TensorFlow/TensorFlow/.venv/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
I0000 00:00:1748054183.369523  251841 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1067 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9


Download and prepare the training data.

In [8]:
(train_images, train_labels), _ = keras.datasets.fashion_mnist.load_data()
train_images = train_images / 255.0

## Train the model and log data

Before training, define the [Keras TensorBoard callback](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard), specifying the log directory. By passing this callback to Model.fit(), you ensure that graph data is logged for visualization in TensorBoard.

In [9]:
# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
#tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=logdir,
    histogram_freq=1,
    write_graph=True,  # Ensure graph writing is enabled
    profile_batch = '10,12',
    update_freq='epoch'
)


# Train the model.
# model.fit(
#     train_images,
#     train_labels, 
#     batch_size=64,
#     epochs=5, 
#     callbacks=[tensorboard_callback])



2025-05-23 22:36:23.905072: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:103] Profiler session initializing.
2025-05-23 22:36:23.905085: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:118] Profiler session started.
I0000 00:00:1748054183.905104  251841 cupti_tracer.cc:1026] Profiler found 1 GPUs
W0000 00:00:1748054183.914977  251841 cupti_tracer.cc:1213] Fail to use per-thread activity buffer, cupti trace overhead may be big. CUPTI ERROR CODE:1
2025-05-23 22:36:23.915077: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:130] Profiler session tear down.
I0000 00:00:1748054183.915130  251841 cupti_tracer.cc:1249] CUPTI activity buffer flushed


In [10]:
# If you're using custom functions with @tf.function, wrap the tracing properly:

# Set up logging for custom functions
writer = tf.summary.create_file_writer(logdir)

# Enable tracing
#tf.summary.trace_on(graph=True, profiler=True)

# Run your model/function
model.fit(
    train_images,
    train_labels, 
    batch_size=64,
    epochs=5, 
    callbacks=[tensorboard_callback])

# Export the trace
with writer.as_default():
    tf.summary.trace_on(graph=True, profiler=True, profiler_outdir=logdir)
    tf.summary.trace_export(
        name="graph",
        step=0,
        profiler_outdir=logdir
    )

Epoch 1/5


I0000 00:00:1748054184.509208  251999 service.cc:152] XLA service 0x795de4016d00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748054184.509224  251999 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2025-05-23 22:36:24.519301: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1748054184.563229  251999 cuda_dnn.cc:529] Loaded cuDNN version 90300


148/938 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4945 - loss: 1.4784

I0000 00:00:1748054185.249909  251999 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2025-05-23 22:36:25.283424: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:103] Profiler session initializing.
2025-05-23 22:36:25.283434: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:118] Profiler session started.
W0000 00:00:1748054185.290974  251841 cupti_tracer.cc:1213] Fail to use per-thread activity buffer, cupti trace overhead may be big. CUPTI ERROR CODE:1
2025-05-23 22:36:25.296469: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:68] Profiler session collecting data.
I0000 00:00:1748054185.298132  251841 cupti_tracer.cc:1249] CUPTI activity buffer flushed
I0000 00:00:1748054185.301608  251841 cupti_collector.cc:793]  GpuTracer has collected 120 callback api events and 110 activity events. 
I0000 00:00:1748054185.301751  251841 cupti_collector.cc:796]  GpuTracer max callback_events: 2097152,

938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6749 - loss: 0.9544
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 1s 908us/step - accuracy: 0.8178 - loss: 0.5161
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 1s 903us/step - accuracy: 0.8327 - loss: 0.4707
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 1s 920us/step - accuracy: 0.8383 - loss: 0.4477
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 1s 894us/step - accuracy: 0.8402 - loss: 0.4391


2025-05-23 22:36:30.161365: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:103] Profiler session initializing.
2025-05-23 22:36:30.161375: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:118] Profiler session started.
W0000 00:00:1748054190.168614  251841 cupti_tracer.cc:1213] Fail to use per-thread activity buffer, cupti trace overhead may be big. CUPTI ERROR CODE:1
2025-05-23 22:36:30.170548: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:68] Profiler session collecting data.
I0000 00:00:1748054190.172973  251841 cupti_tracer.cc:1249] CUPTI activity buffer flushed
I0000 00:00:1748054190.177266  251841 cupti_collector.cc:793]  GpuTracer has collected 3 callback api events and 2 activity events. 
I0000 00:00:1748054190.177392  251841 cupti_collector.cc:796]  GpuTracer max callback_events: 2097152, max activity events: 2097152
2025-05-23 22:36:30.178490: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:130] Profiler session tear down.
2025-05-23 

## Op-level graph

Start TensorBoard and wait a few seconds for the UI to load. Select the Graphs dashboard by tapping “Graphs” at the top. 

In [13]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6011 (pid 252536), started 0:00:11 ago. (Use '!kill 252536' to kill it.)

By default, TensorBoard displays the **op-level graph**. (On the left, you can see the “Default” tag selected.)  Note that the graph is inverted; data flows from bottom to top, so it’s upside down compared to the code. However, you can see that the graph closely matches the Keras model definition, with extra edges to other computation nodes.

Graphs are often very large, so you can manipulate the graph visualization:

* Scroll to **zoom** in and out
* Drag to **pan**
* Double clicking toggles **node expansion** (a node can be a container for other nodes)

You can also see metadata by clicking on a node. This allows you to see inputs, outputs, shapes and other details.


<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/graphs_computation.png?raw=1"/> -->

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/graphs_computation_detail.png?raw=1"/> -->

## Conceptual graph

In addition to the execution graph, TensorBoard also displays a **conceptual graph**. This is a view of just the Keras model. This may be useful if you’re reusing a saved model and you want to examine or validate its structure.

To see the conceptual graph, select the “keras” tag. For this example, you’ll see a collapsed **Sequential** node. Double-click the node to see the model’s structure:

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/graphs_tag_selection.png?raw=1"/> --> <br/>
<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/graphs_conceptual.png?raw=1"/> -->

## Graphs of tf.functions

The examples so far have described graphs of Keras models, where the graphs have been created by defining Keras layers and calling Model.fit().

You may encounter a situation where you need to use the `tf.function` annotation to ["autograph"](https://www.tensorflow.org/guide/function), i.e., transform, a Python computation function into a high-performance TensorFlow graph. For these situations, you use **TensorFlow Summary Trace API** to log autographed functions for visualization in TensorBoard.

To use the Summary Trace API:

*   Define and annotate a function with `tf.function`
*   Use `tf.summary.trace_on()` immediately before your function call site.
*    Add profile information (memory, CPU time) to graph by passing `profiler=True`
*   With a Summary file writer, call `tf.summary.trace_export()` to save the log data

You can then use TensorBoard to see how your function behaves.


In [12]:
# The function to be traced.
@tf.function
def my_func(x, y):
  # A simple hand-rolled layer.
  return tf.nn.relu(tf.matmul(x, y))

# Set up logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/func/%s' % stamp
writer = tf.summary.create_file_writer(logdir)

# Sample data for your function.
x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True, profiler=True)
# Call only one tf.function when tracing.
z = my_func(x, y)
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)

UnavailableError: Cannot export profiling results. No profiler is running.

In [ ]:
%tensorboard --logdir logs/func

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/graphs_autograph.png?raw=1"/> -->

You can now see the structure of your function as understood by TensorBoard. Click on the "Profile" radiobutton to see CPU and memory statistics.